### 从一个文件夹里读取csv(每个交易品种一个csv)，拼成一个dataframe(按照datetime对齐)，然后计算收益率

#### 获取并过滤文件夹下的文件名

In [1]:
import pandas as pd
import numpy as np
import time
import os 

# 读某个文件夹下的文件名(os.listdir())，返回一个名字的list
path ='/Users/yuxun/python/multifactor/source2/'
files = os.listdir(path)
print('All files:')
print(files)

# 过滤得到想要的文件名
### 可以用list(filter(func, olist))的方式过滤，但如果仅涉及简单的逻辑判断（也是很多情况），用列表解析是比较好的
### [i for i in oldlist if (...)]
### [i if (...) else (...) for i in oldlist]
# files_csv = list(filter(lambda x:x[-4:]=='.csv', files))
files_csv = [i for i in files if i[-4:] == '.csv']
path = [path+i for i in files_csv]

print('Filtered files:')
print(files_csv)

All files:
['BNB_USDT.csv', 'BCH_USDT.csv', 'TUSD_USDT.csv', 'ICX_USDT.csv', 'ADA_USDT.csv', 'VEN_USDT.csv', 'QTUM_USDT.csv', 'TRX_USDT.csv', 'LTC_USDT.csv', 'XRP_USDT.csv', 'BTC_USDT.csv', 'ONT_USDT.csv', 'XLM_USDT.csv', 'IOTA_USDT.csv', 'EOS_USDT.csv', 'ETH_USDT.csv', 'NEO_USDT.csv', 'NULS_USDT.csv', 'ETC_USDT.csv']
Filtered files:
['BNB_USDT.csv', 'BCH_USDT.csv', 'TUSD_USDT.csv', 'ICX_USDT.csv', 'ADA_USDT.csv', 'VEN_USDT.csv', 'QTUM_USDT.csv', 'TRX_USDT.csv', 'LTC_USDT.csv', 'XRP_USDT.csv', 'BTC_USDT.csv', 'ONT_USDT.csv', 'XLM_USDT.csv', 'IOTA_USDT.csv', 'EOS_USDT.csv', 'ETH_USDT.csv', 'NEO_USDT.csv', 'NULS_USDT.csv', 'ETC_USDT.csv']


#### 读取csv生成dataframe，并merge数据

In [2]:
# 获取symbol
symbols = [x[:-4] for x in files_csv]
# data是一个字典，key是symbol，value是pd.read_csv()得到的dataframe
data = {}
for i in range(len(symbols)):
    data[symbols[i]] = pd.read_csv(path[i])
    
# 生成DataFrame
### 生成一个初始化的DataFrame
### 注意，由于需要datetime对齐，所以每个dataframe都必须以datetime作为index(或者至少保留这一列信息)
timestamp =  [time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(i/1000)) for i in data['BTC_USDT'].timestamp]
dt_close = data['BTC_USDT'][['Close']].rename(columns={'Close': 'BTC'})
dt_close.index = timestamp
### 循环读数据，并且进行merge操作(以index为依据)
for key, value in data.items():  # 字典的for循环：dict.items()可以拿到字典的所有k-v对
    if key != 'BTC_USDT':
        timestamp = [time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(i/1000)) for i in data[key].timestamp]
        dt_close_temp = data[key][['Close']].rename(columns={'Close': key.split(sep='_')[0]})
        dt_close_temp.index = timestamp
        dt_close = dt_close.merge(dt_close_temp, how='outer', left_index=True, right_index=True)
        
print(dt_close)

                         BTC      BNB     BCH    TUSD     ICX      ADA  \
2017-08-17 08:00:00  4285.08      NaN     NaN     NaN     NaN      NaN   
2017-08-18 08:00:00  4108.37      NaN     NaN     NaN     NaN      NaN   
2017-08-19 08:00:00  4139.98      NaN     NaN     NaN     NaN      NaN   
2017-08-20 08:00:00  4086.29      NaN     NaN     NaN     NaN      NaN   
2017-08-21 08:00:00  4016.00      NaN     NaN     NaN     NaN      NaN   
2017-08-22 08:00:00  4040.00      NaN     NaN     NaN     NaN      NaN   
2017-08-23 08:00:00  4114.01      NaN     NaN     NaN     NaN      NaN   
2017-08-24 08:00:00  4316.01      NaN     NaN     NaN     NaN      NaN   
2017-08-25 08:00:00  4280.68      NaN     NaN     NaN     NaN      NaN   
2017-08-26 08:00:00  4337.44      NaN     NaN     NaN     NaN      NaN   
2017-08-27 08:00:00  4310.01      NaN     NaN     NaN     NaN      NaN   
2017-08-28 08:00:00  4386.69      NaN     NaN     NaN     NaN      NaN   
2017-08-29 08:00:00  4587.48      NaN 

#### 用apply函数计算对数收益率

In [3]:
# 收益率计算
### 用到了pandas的apply函数，是对一整列dataframe数据操作的很好用的函数，这里的x可以认为是一列
### 注意到，实际上x.shift(n)，当n>0时这个式子是在计算过去n天的累计收益(n<0且lambda函数反过来，就是未来n天的收益了)
dt_return = dt_close.apply(lambda x: np.log(x) - np.log(x.shift(1)))
print(dt_return)

                          BTC       BNB       BCH      TUSD       ICX  \
2017-08-17 08:00:00       NaN       NaN       NaN       NaN       NaN   
2017-08-18 08:00:00 -0.042113       NaN       NaN       NaN       NaN   
2017-08-19 08:00:00  0.007665       NaN       NaN       NaN       NaN   
2017-08-20 08:00:00 -0.013053       NaN       NaN       NaN       NaN   
2017-08-21 08:00:00 -0.017351       NaN       NaN       NaN       NaN   
2017-08-22 08:00:00  0.005958       NaN       NaN       NaN       NaN   
2017-08-23 08:00:00  0.018154       NaN       NaN       NaN       NaN   
2017-08-24 08:00:00  0.047933       NaN       NaN       NaN       NaN   
2017-08-25 08:00:00 -0.008219       NaN       NaN       NaN       NaN   
2017-08-26 08:00:00  0.013172       NaN       NaN       NaN       NaN   
2017-08-27 08:00:00 -0.006344       NaN       NaN       NaN       NaN   
2017-08-28 08:00:00  0.017635       NaN       NaN       NaN       NaN   
2017-08-29 08:00:00  0.044756       NaN       NaN  

In [4]:
# 未来5日收益率计算
BTC = dt_close[['BTC']]
BTC['BTC_fR_0_5'] = BTC.apply(lambda x: np.log(x.shift(-5)) - np.log(x))
print(BTC)

                         BTC  BTC_fR_0_5
2017-08-17 08:00:00  4285.08   -0.058895
2017-08-18 08:00:00  4108.37    0.001372
2017-08-19 08:00:00  4139.98    0.041640
2017-08-20 08:00:00  4086.29    0.046474
2017-08-21 08:00:00  4016.00    0.076998
2017-08-22 08:00:00  4040.00    0.064696
2017-08-23 08:00:00  4114.01    0.064177
2017-08-24 08:00:00  4316.01    0.060999
2017-08-25 08:00:00  4280.68    0.062144
2017-08-26 08:00:00  4337.44    0.085560
2017-08-27 08:00:00  4310.01    0.114922
2017-08-28 08:00:00  4386.69    0.019292
2017-08-29 08:00:00  4587.48   -0.017238
2017-08-30 08:00:00  4555.14   -0.105242
2017-08-31 08:00:00  4724.89   -0.078889
2017-09-01 08:00:00  4834.91   -0.045518
2017-09-02 08:00:00  4472.14    0.047909
2017-09-03 08:00:00  4509.08   -0.051486
2017-09-04 08:00:00  4100.11    0.037976
2017-09-05 08:00:00  4366.47   -0.055588
2017-09-06 08:00:00  4619.77   -0.093246
2017-09-07 08:00:00  4691.61   -0.119367
2017-09-08 08:00:00  4282.80   -0.082237
2017-09-09 08:00

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
